In [1]:
# !pip install catboost
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# !pip install sweetviz
import sweetviz as sv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
rc('font',family= 'malgun gothic')
rc('axes', unicode_minus = False)

import math
%matplotlib inline
import seaborn as sns
# !pip install -U klib
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm, tqdm_notebook
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

C:\Users\inuya\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# 학습, 평가데이터 불러오기

train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding = 'cp949')
target = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding = 'cp949').group

In [3]:
# 같이 전처리 하기 위해서 train과 test를 합침

data = pd.concat([train, test],ignore_index= True)

In [4]:
unique_custid = pd.DataFrame(data.custid.unique()).rename(columns = {0 : 'custid'})

# Preprocessing

In [5]:
# 시간데이터 처리

def hour_process(x):
    x = str(x)
    if len(x) == 4:
        return x[:2]
    elif len(x) == 3:
        return x[:1]
    elif len(x) == 2:
        return '0'

data['sales_hour'] = data['sales_time'].apply(hour_process)

def minute_process(x):
    x = str(x)
    if len(x) == 4:
        return x[2:]
    elif len(x) == 3:
        return x[1:]
    elif len(x) == 2:
        return x
    
data['sales_minute'] = data['sales_time'].apply(minute_process)


data['sales_hour'] = data['sales_hour'].astype(int)
data['sales_minute'] = data['sales_minute'].astype(int)

In [6]:
# 날짜데이터 처리

data['sales_month_origin'] = data['sales_month']
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)
data['year'] = data['sales_month_origin'].apply(lambda x: 2018 if x>12 else 2017)

data['year_month_day'] = data['year'].astype(str) + '_' + data['sales_month'].astype(str) + '_' +\
                                data['sales_day'].astype(str) + '_' + data['sales_time'].astype(str)

data['datetime'] = pd.to_datetime(data['year_month_day'], format = '%Y_%m_%d_%H%M')


data['diff_time'] = data.groupby('custid')['datetime'].diff().fillna('00:00:00').astype(str)
data['diff_time'] = data['diff_time'].str.split(' days').apply(lambda x: 0 if x[0] == '00:00:00' else x[0]).astype(int)

In [7]:
# corner_nm 에서 겹치는 값들 처리하기

data.loc[data.corner_nm == '모피.피혁', 'corner_nm'] = '모피/피혁'
data.loc[data.corner_nm == '원목(주니어)', 'corner_nm'] = '원목/주니어'
data.loc[data.corner_nm == '우산,장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '우산장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '트.단품 ', 'corner_nm'] = '트단품'
data.loc[data.corner_nm == 'TV,VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'TV.VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'GBR  지원', 'corner_nm'] = 'GBR지원'
data.loc[data.corner_nm == '페레  지원', 'corner_nm'] = '페레지원'
data.loc[data.corner_nm == '라디오.카세트', 'corner_nm'] = '라디오/카세트'
data.loc[data.corner_nm == '스포츠용퓸', 'corner_nm'] = '스포츠용품'
data.loc[data.corner_nm == '카세트,전화기', 'corner_nm'] = '전화기/카세트'

In [8]:
# pc_nm 에서 겹치는 값들 처리하기

data.loc[data.pc_nm == '침구,수예', 'pc_nm'] = '침구/수예'
data.loc[data.pc_nm == '디자이너부띠크', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '디자이너부틱', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '니트,단품,모피', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '니트/단품', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '로얄부틱', 'pc_nm'] = '로얄부띠끄'
data.loc[data.pc_nm == '트랜디 케쥬얼', 'pc_nm'] = '트랜디캐쥬얼'

In [9]:
# lgbm 에서 json 오류 방지를 위해 특수문자 , 제거

import re
data.rename(columns = lambda x:re.sub(',', '/', x), inplace = True)

# Categoric Feature Making

- **corner_nm**

In [10]:
IDtest = test.custid.unique()

In [11]:
level = 'corner_nm'
data[level].nunique()

300

In [12]:
def where(x):
    if len(x) > 1:
        result = 1
    else:
        result = 0
    return result

In [13]:
catFeatures_train_cor = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest').\
                        drop(columns=['custid']).values
catFeatures_test_cor = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

# max_features = X_train_cat.shape[1]

In [14]:
catFeatures_train_cor = pd.DataFrame(catFeatures_train_cor)
catFeatures_test_cor = pd.DataFrame(catFeatures_test_cor)

In [15]:
catFeatures_train_cor.columns = catFeatures_train_cor.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_cor.columns = catFeatures_test_cor.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

- **brd_nm**

In [16]:
level = 'brd_nm'
data[level].nunique()

1873

In [17]:
catFeatures_train_brd = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_brd = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [18]:
catFeatures_train_brd = pd.DataFrame(catFeatures_train_brd)
catFeatures_test_brd = pd.DataFrame(catFeatures_test_brd)

In [19]:
catFeatures_train_brd.columns = catFeatures_train_brd.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_brd.columns = catFeatures_test_brd.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

- **pc_nm**

In [20]:
level = 'pc_nm'
data[level].nunique()

72

In [21]:
catFeatures_train_pc = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_pc = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [22]:
catFeatures_train_pc = pd.DataFrame(catFeatures_train_pc)
catFeatures_test_pc = pd.DataFrame(catFeatures_test_pc)

In [23]:
catFeatures_train_pc.columns = catFeatures_train_pc.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_pc.columns = catFeatures_test_pc.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

- **part_nm**-

In [24]:
level = 'part_nm'
data[level].nunique()

30

In [25]:
catFeatures_train_part = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_part = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [26]:
catFeatures_train_part = pd.DataFrame(catFeatures_train_part)
catFeatures_test_part = pd.DataFrame(catFeatures_test_part)

In [27]:
catFeatures_train_part.columns = catFeatures_train_part.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_part.columns = catFeatures_test_part.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[team_nm]** 

In [28]:
level = 'team_nm'
data[level].nunique()

4

In [29]:
catFeatures_train_team = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                        drop(columns=['custid']).values

catFeatures_test_team = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                        drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [30]:
catFeatures_train_team = pd.DataFrame(catFeatures_train_team)
catFeatures_test_team = pd.DataFrame(catFeatures_test_team)

In [31]:
catFeatures_train_team.columns = catFeatures_train_team.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_team.columns = catFeatures_test_team.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[buyer_nm]**

In [32]:
level = 'buyer_nm'
data[level].nunique()

35

In [33]:
catFeatures_train_buyer = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                        drop(columns=['custid']).values

catFeatures_test_buyer = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: where(x), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                        drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [34]:
catFeatures_train_buyer = pd.DataFrame(catFeatures_train_buyer)
catFeatures_test_buyer = pd.DataFrame(catFeatures_test_buyer)

In [35]:
catFeatures_train_buyer.columns = catFeatures_train_buyer.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_buyer.columns = catFeatures_test_buyer.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

# Merge Categoric Features

In [36]:
onehot_features_train = pd.concat([catFeatures_train_cor, catFeatures_train_brd, catFeatures_train_pc,\
                            catFeatures_train_part, catFeatures_train_team, catFeatures_train_buyer], axis=1)

In [37]:
onehot_features_test = pd.concat([catFeatures_test_cor, catFeatures_test_brd, catFeatures_test_pc, \
                           catFeatures_test_part, catFeatures_test_team, catFeatures_test_buyer], axis=1)

In [38]:
onehot_features_train.head()

,corner_nm_0_onehot,corner_nm_1_onehot,corner_nm_2_onehot,corner_nm_3_onehot,corner_nm_4_onehot,corner_nm_5_onehot,corner_nm_6_onehot,corner_nm_7_onehot,corner_nm_8_onehot,corner_nm_9_onehot,...,buyer_nm_25_onehot,buyer_nm_26_onehot,buyer_nm_27_onehot,buyer_nm_28_onehot,buyer_nm_29_onehot,buyer_nm_30_onehot,buyer_nm_31_onehot,buyer_nm_32_onehot,buyer_nm_33_onehot,buyer_nm_34_onehot
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [39]:
onehot_features_train.to_csv('seo_onehot_features_train.csv', index=False,encoding = 'utf-8')
onehot_features_test.to_csv('seo_onehot_features_test.csv', index=False,encoding = 'utf-8')